
# `Материалы кафедры ММП факультета ВМК МГУ. Введение в глубокое обучение.`

# `Леуция 07. Рекуррентные нейронные сети`

### `Материалы составил Алексеев Илья`

#### `Москва, Весенний семестр 2025`

[Советуем так же ознамиться с материалами предыдущих лет](https://github.com/mmp-practicum-team/mmp_practicum_spring_2024/blob/main/Seminars/Seminar%2006/%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B5%20%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%A1%D0%B5%D1%82%D0%B8.ipynb)


## Вспомним w2v


На прошлой неделе вы познакомились с языковой моделью word2vec. Это модель получения эмбедингов для отдельных токенов. Её можно применять как для слов, так и для текстов. Чтобы получить эмбединг текста с помощью word2vec, достаточно усреднить эмбединги токенов, входящих в текст. Схематически это выглядит следующим образом:

![](notes.assets/rnn-w2v.drawio.svg)

Такую модель можно даже дообучать, чтобы адаптировать word2vec эмбединги под конкретную задачу. Однако у данной модели есть существенный недостаток. Рассмотрим пример предложения: “Робин съел лук”. Слово “лук” — известный пример омонимии. Смысл этого токена нельзя понять без контекста, в котором он встретился. В данном случае, скорее всего, это овощ. Но эмбединг слова “лук”, полученный методом word2vec, будет передавать информацию обо всех значениях этого токена, а не только ту её узкую и более точную часть, которая относится к текущему контексту.

## RNN Cell


Введем модель, которую назовем рекуррентной ячейкой:

![image-20250316131041975](./notes.assets/rnn-rnn-cell.drawio.svg)

Рекуррентная ячейка имеет два входа $h,x$ и два выхода $h',y$:
$$
h'=g_h(W_1x+W_2h),\quad
y=g_y(W_3h'),
$$
где $g_h(\cdot),g_y(\cdot)$ — функции активации. Этот функциональный элемент действует очень похоже на полносвязную сеть, только помимо потока “вперед”, в ней реализован поток “вправо”.
$$
h',y=\text{RNNCell}(h,x).
$$
Рекуррентная ячейка “смешивает” векторы $h$ и $x$.

## RNN


Применим рекуррентную ячейку к целому тексту путем обработки всех токенов одной и той же рекуррентной ячейкой последовательно:

![image-20250316131756463](./notes.assets/rnn-rnn.drawio.svg)
$$
h_t,y_t=\text{RNNCell}(h_{t-1},x_t),\quad t=\overline{1,T}.
$$
Векторы $h_t$ будем называть скрытыми состояниями. Вектор $h_0$ инициализируется нулевым. Совершая проход по предложению, рекуррентная сеть сохраняет в $h$ информацию о всех токенах. А именно, в $h_t$ “примешана” информация о всех токенах $x_1,\ldots,x_t$.

Самое главное, что в результате такого прогона получается последовательность $y_1,\ldots,y_T$ — эти векторы можно воспринимать как репрезентации токенов $x_1,\ldots, x_T$, обогащенные контекстной информацией.

## Применение и обучение RNN


Необходимо разобраться, как обучать рекуррентную сеть таким образом, чтобы в контекст $h$ и в обновленные репрезентации $y$ записывалась информация, полезная для решения конкретных задач NLP.

### Классификация


Чтобы обучить рекуррентную сеть на задачу классификации, можно применить усреднение обновленных репрезентаций:

![image-20250316133313712](./notes.assets/rnn-clf.drawio.svg)

В данной схеме все операции дифференцируемы, поэтому обучая классификационную голову на кросс-энтропийный лосс, мы получим градиенты и для рекуррентного блока. Заметим, что поскольку везде применяется одна и та же рекуррентная ячейка, то обучаемыми параметрами являются только 5 матриц: $W_1,W_2,W_3,W_4$ и матрица в классификационной голове. Это сближает рекуррентные сети со сверточными в том смысле, что обе архитектуры основаны на идее применения одних и тех же (разделенных) параметров к разным частям входа.

### Causal Language Modeling


Или задача предсказания следующего токена. Это генеративная задача. Сначала объясним, как это должно работать, и только потом, как это обучать.

На вход такой задаче приходит начало предложения, которое называют затравкой (prompt). Задача продолжить это предложение. Допустим, в качестве затравки используется “Однажды я съездил в Питер и со мной случилось следующее:”, тогда задача модели сгенерировать текст, который может выдумать, что с ней случилось в Питере.

Рекуррентные сети справляются с такими задачами путем авторегрессионного инференса:

![image-20250316134110796](./notes.assets/rnn-clm.drawio.svg)

На этой схеме классификатор везде один и тот же и он выдает вероятности по всем токенам в вокабулярии. Это вероятности каждого токена быть следующим в предложении. Предсказав токен $e_{T+1}$, его можно подать на вход рекуррентной ячейке (и соответственно классификатору), которая предскажет токен $e_{T+2}$. И так далее.

Термин “авторегрессия” взят из задачи предсказания временных рядов, где тоже часто используют свои предыдущие предсказания для получения все новых и новых.

Остается вопрос: как обучить рекуррентную сеть для работы в авторегрессионном режиме? Для этого достаточно собрать большой корпус текстов. Один обучающий пример формируется следующим образом. Допустим у нас есть текст $x_1,\ldots,x_T$. Тогда

1. прогоним через RNN текст $w_1,\ldots,w_{T-1}$ и получим $y_1,\ldots,y_{T-1}$
2. пропустим игреки через классификатор и получим логиты
3. используем кросс-энтропийный лосс, где истинными метками будут  $w_2,\ldots,w_T$ 

В следующих занятиях будет рассказано больше практических соображений о том, как формировать обучающую выборку и батчи для обучения языковой модели для генерации текстов.

## Распознавание именованных сущностей


Именованными сущностями (named entity) называют адрес, имя, время и прочее. Обычно они представляют некоторую информацию, которую нужно извлечь из текста. Попытаемся пойти от простого к сложному. Допустим, мы хотим извлекать человеческие имена вроде Илья, Петя, Гриша. Самый простой способ — это бинарная классификация каждого токена на предмет того, является ли он человеческим именем:

![image-20250316145559488](./notes.assets/rnn-ner.drawio.svg)

Чтобы обучить такую модель, достаточно собрать корпус текстов и сделать разметку для каждого токена.

Рассмотрим пример сложнее: распознавание адресов вроде “ул. Строителей, дом 9”. Главная сложность в том, что  именованная сущность распростерлась на несколько токенов, каждый из которых не является именованной сущностью. В таких случаях вместо бинарной классификации используют классификацию на три класса: beginning, inside, outside (так называемая BIO-разметка). Каждый токен мы классифицируем на три класса:

- B: первый токен именованной сущности
- I: токен, относящийся к именованной сущности, но не являющийся первым
- O: токен, не относящийся к именованной сущности

Рассмотрим пример еще сложнее: распознавание и человеческих имен, и адресов. Сложность в том, что нужно уметь извлекать разные сущности с помощью одной модели. Решение простое: заведем BIO классы для каждой сущности.

### Перевод


RNN можно применять для перевода с языка на язык (например, с английского на французский). Для этого используются так называемые автокодровщики, с идеей которых вы можете быть знакомы, если помните как решались img-to-img задачи. Сейчас мы будем решать задачу text-to-text.

Допустим, мы хотим обучить модель для перевода с английского на французский. На инференсе должно работать в следующем режиме:

- прогоним английский текст $x_1,\ldots,x_T$ через RNN и получим $h_T$ — скрытое состояние RNN после токена $x_T$
- возьмем вторую RNN сеть, которая будет генерировать перевод в авторегрессионном режиме
- дли этого подадим ей в качестве затравки служебный токен `[BOS]` (означает beginning of sequence), и в качестве начального вектора контекста $h_T$
- генерируем перевод до тех пор, пока языковая голова не предскажет токен `[EOS]` (означает end of sequence)

Чтобы обучить такую модель достаточно применить тот же рецепт, что и для обучения causal languge model.

Первую RNN сеть (которая кодирует $x_1,\ldots,x_T$ в вектор $h_T$) называют кодировщиком. Вторую RNN сеть (которая из $h_T$ генерирует перевод в авторегрессионном режиме) называют декодировщиком.

![image-20250316152710814](./notes.assets/rnn-translation.drawio.svg)

## Модификации RNN


Очевидный недостаток обсуждаемой модели в том, что рекуррентное движение происходит только слева направо. Последние токены предложения “видят” первые, но не наоборот. Чтобы исправить это, достаточно добавить вторую RNN сеть, которая будет совершать свой путь в обратном направлении. Тогда в качестве обновленной репрезентации токена $x_t$ можно взять $(y_t+\tilde y_t)$, где $y_t$ — выход из первой сети, а $\tilde y_t$ — выход из второй.

![](notes.assets/rnn-bidirectional.drawio.svg) 



Второй очевидной модификаций является многослойная RNN сеть. Каждый слой будет выполнять ту же функцию что и в MLP и в свёрточных сетях: генерация все более высокоуровневых и полезных для задачи признаков.

![](notes.assets/rnn-multilayer.drawio.svg)

Еще одна модификация, которая может сработать, это прокидывание остаточных связей между слоями.

![](notes.assets/rnn-skip-connection.drawio.svg)

## Регуляризация в RNN


Обучать RNN довольно тяжело на длинных последовательностях из-за угасания и взрыва градиентов. Для борьбы с этим существует стандартный набор трюков: нормализация, дропаут, правильная токенизация. Однако со своими нюансами.

### LayerNorm


Во-первых, для текстов используют не batch-нормализацию, а layer-нормализацию. 

Вход $x$ представляет собой тензор размера `(batch_size, sequence_length, hidden_size)`. Батч-норм считает статистики вдоль размерности батча для каждого признака каждого токена

$$
\mu_{h} = \frac{1}{BT} \sum_{i,t=1}^{B,T} x_{ith}\\
\sigma_{h}^2 = \frac{1}{BT} \sum_{i,t=1}^{B,T} (x_{ith} - \mu_{h})^2\\
y_{ith} = \gamma_h\frac{x_{ith} - \mu_{h}}{\sqrt{\sigma_{h}^2 + \epsilon}}+\delta_h
$$
В задачах с текстом обычно батч небольшой, а тексты имеют разную длину, поэтому статистики посчитанные таким образом могут быть неточными.

Layer-нормализация считает статистики вдоль размерности признака, поэтому лишена этих недостатков:

$$
\mu_{it} = \frac{1}{H} \sum_{h=1}^H x_{ith}\\
\sigma_{it}^2 = \frac{1}{H} \sum_{h=1}^H (x_{ith} - \mu_{it})^2\\
y_{ith} = \gamma_h\frac{x_{ith} - \mu_{it}}{\sqrt{\sigma_{it}^2 + \epsilon}}+\delta_h
$$

### DropOut


С этим вы хорошо познакомитесь в домашке. 

### Токенизация


Очевидно, что чем длиннее входная последовательность, тем сложнее решать задачи с ней. На длину последовательности влияет способ токенизации текста. Рассмотрим два крайних случая

**Токен = символ.**

- плюс: маленький вокабулярий (=число поддерживаемых символов) => нужно относительно мало выходов классификатора 
- минус: вход относительно длинный

**Токен = слово.**

- плюс: вход относительно короткий
- минус: огромный вокабулярий который все равно не охватит все слова языка

Балансом между этими крайними случаями является так называемые BPE-токенизация. 

---

**Алгоритм:**

1. **Инициализация:**

   - Пусть текст $ T $ разбит на слова $ W = \{w_1, w_2, \dots, w_n\} $.
   - Разбей каждое слово $ w_i $ на символы: $ w_i = \{c_1, c_2, \dots, c_m\} $.
   - Построй частотный словарь $ D $, где $ D[c_j] $ — частота символа $ c_j $.

2. **Обучение:**
   - Повторяй до достижения желаемого размера словаря $ |D| $:
     - Найди наиболее частую пару $ (x, y) $ в $ D $:
       $$
       (x, y) = \arg\max_{(x, y) \in D} \text{count}(x, y)
       $$
     - Создай новый субтокен $ z = x + y $.
     - Обнови словарь $ D $:
       $$
       D[z] = \text{count}(x, y)\\
       D[x] = D[x] - \text{count}(x, y)\\
       D[y] = D[y] - \text{count}(x, y)
       $$
     - Удали пару $ (x, y) $ из словаря, если их частота стала нулевой.

3. **Токенизация:**

   - Для каждого слова $ w_i $ в тексте $ T $:
     - Разбей $ w_i $ на символы: $ w_i = \{c_1, c_2, \dots, c_m\} $.
     - Итерируйся по символам, объединяя их в субтокены, используя словарь $ D $:
       $$
       \text{tokenize}(w_i) = \arg\max_{z \in D} \text{count}(z)
       $$
     - Если токен не найден, используй специальный токен (например, `<unk>`).

**Пример:**

Пусть текст: "low lower lowest".

1. Инициализация:
   - Слова: ["low", "lower", "lowest"].
   - Частотный словарь: {"l": 3, "o": 3, "w": 3, "e": 2, "r": 1, "s": 1, "t": 1}.

2. Обучение:
   - Наиболее частая пара: "l" и "o" → "lo".
   - Новый словарь: {"lo": 3, "w": 3, "e": 2, "r": 1, "s": 1, "t": 1}.
   - Следующая пара: "lo" и "w" → "low".
   - Новый словарь: {"low": 3, "e": 2, "r": 1, "s": 1, "t": 1}.

3. Токенизация:
   - "low" → ["low"].
   - "lower" → ["low", "e", "r"].
   - "lowest" → ["low", "e", "s", "t"].

---

## Механизм внимания


Обратимся к задаче перевода (text-to-text). У предложенного нами решения с помощью RNN есть очевидный недостаток: во время авторегрессионного предсказания перед декодером стоит слишком сложная задача сгенерировать целый текст имея на вход всего лишь супер сжатое представление исходного текста в виде одного вектора. Что если позволить декодеру в каждый момент генерации видеть не только вектор контекста $h$, но и эмбединги исходной последовательности? Тогда декодер смог бы совершать перевод например так, как это делают люди: смотря на конкретную часть исходного текста и подбирая слова для перевода. Такую возможность дает механизм внимания.

Посмотрим внимательно на формулу RNN ячейки:
$$
s_t=g_s(W_1z_t+W_2s_{t-1}),\quad y_t=g_y(W_3s_t).
$$
Активация $y_t$ используется как признаки для предсказания следующего токена $z_{t+1}$. При этом $y_t$ получается только из текущего токена перевода $z_t$ и из вектора контекста $s_{t-1}$. Что если мы модифицируем сумму в скобках:
$$
s_t=g_s(W_1z_t+W_2s_{t-1}+W_3d_t),\quad d_t=\sum_{i=1}^Th_i\alpha_{it},
$$
где $\alpha_{it}$ — некоторые веса, характеризующие полезность $i$-го токена исходной последовательности для предсказания $(t+1)$-го токена перевода. Если мы сможем для каждого шага $t$ подбирать нужные веса, получится реализовать умную агрегацию информации из исходной последовательности, которую можно назвать механизмом внимания (токены перевода уделяют разное внимание исходным токенам).

Простейший вариант определения весов $\alpha$ следующий:
$$
\alpha_{it}={\exp(\text{sim}(s_{t-1},h_i))\over\sum_{j=1}^T\exp(\text{sim}(s_{t-1},h_j))},\quad\text{sim}(x,y)=\langle x,y\rangle.
$$
![Explainable AI: Visualizing Attention in Transformers](./notes.assets/Screen-Shot-2023-07-11-at-5.43.47-PM.png)

## LSTM


Используется два вектора внутреннего состояния: $h_t$ — ближайший контекст (короткая память), $c_t$ — контекст всего предложения (долгая память).

Вычисляются следующие величины:

- forget gate: $f = \sigma(h_{t-1}W^f + x_t U^f+b_f)\in(0,1)$ — регулирует, какие сигналы (компоненты $c_t$) нужно ослабить (механизм стирания из памяти)
- input gate: $i = \sigma(h_{t-1}W^i + x_t U^i+b_i)\in(0,1)$ — регулирует, нужно ли добавить что-то в долговременную память
- output gate: $o = \sigma(h_{t-1}W^o + x_t U^o+b_o)\in(0,1)$ — регулирует, насколько долговременную память $c_t$ уместно использовать прямо сейчас
- активация прямо как до этого: $g=\tanh(h_{t-1} W^g + x_t U^g+b_g)$

Из них формируются $h_t$ и $c_t$:
$$
c_t = f \odot c_{t-1} +  i \odot  g,\quad
h_t =  o \odot \tanh(c_t)
$$
Вектор $h_t$ используется одновременно и как выход, и как внутреннее состояние.

<img src="notes.assets/lstm.png" style="zoom:67%;" />

Заметим, что в LSTM решена проблема затухания градиента (см. выражение для обновления $c_t$).

## GRU (gated recurrent units)


Нет разделения на долго- и кратковременную память, используется один вектор состояния $h_t$. Вычисления:

- reset gate: $r_t=\sigma(W^rh_{t-1}+U^r x_t+b^r)\in(0,1)$ — механизм стирания из памяти
- update gate: $u_t=\sigma(W^uh_{t-1}+U^ux_t+b^u)\in(0,1)$ — механизм записи в память (комбинирует в себе input и output gate одновременно)
- активация: $g_t=g(W^g(r_t\odot h_{t-1})+U^gx_t+b^g)$
- скрытое состояние: $h_t=u_t\odot h_{t-1}+(1-u_t)\odot g_t$

<img src="notes.assets/gru.svg" style="zoom:67%;" />